# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# Dataset loading

In [3]:
dataset_name='pokemon'

In [4]:
IMG_DIR = 'C:\\projects\\pokemon\DS06\\'
VAL_PCT = 10
VAL_FORMAT = 'png'

In [5]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [6]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [7]:
inputs_shape= IMG_SIZE=(64, 64, 3)
BATCH_SIZE = 32
latent_dim = 100
intermediate_dim = 50
TRAIN_BUF = 600
TEST_BUF = 100

In [8]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode=None
)

  INFO     | Found 738 training files


  INFO     | Found 71 validation files


In [9]:
import tensorflow as tf

In [10]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)


In [11]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [12]:
_instance_scale

1.0

In [13]:
import numpy as np
from collections.abc import Iterable

In [14]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [15]:
_outputs_shape

12288

# Model's Layers definition

In [16]:
_uf=100
c=16
menc_lays = [
    tf.keras.layers.Conv2D(filters=_uf, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Conv2D(filters=_uf*9, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latent_dim)
]

venc_lays = [
    tf.keras.layers.Conv2D(filters=_uf, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Conv2D(filters=_uf*9, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latent_dim)
]

dec_lays = [
    tf.keras.layers.Dense(units=_uf*c*c, activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(c , c, _uf)),
    tf.keras.layers.Conv2DTranspose(filters=_uf, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    tf.keras.layers.Conv2DTranspose(filters=_uf*3, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=3, strides=(1, 1), padding="SAME")
]

# Model definition

In [17]:
model_name = dataset_name+'ConvVAEpsnr'
recoding_dir='..'+sep_local+'..'+sep_local+'recoding'+sep_local+model_name

In [18]:
from training.traditional.autoencoders.VAE import VAE as AE

In [19]:
inputs_shape=IMG_SIZE

In [20]:
variables_params = \
[
    {
        'name': 'encoder_mean', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': menc_lays
    }
    ,
    {
        'name': 'encoder_logvar', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': venc_lays
    }
    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [21]:
from utils.data_and_files.file_utils import create_if_not_exist

In [22]:
_restore = os.path.join(recoding_dir, 'var_save_dir')

In [23]:
create_if_not_exist(_restore)
_restore

'..\\..\\recoding\\pokemonConvVAEpsnr\\var_save_dir'

In [24]:
#to restore trained model, set filepath=_restore

In [25]:
ae = AE( 
    name=model_name,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None
    )

Model: "encoder_mean"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_mean_inputs (InputLa [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 31, 31, 100)       2800      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 15, 900)       810900    
_________________________________________________________________
flatten (Flatten)            (None, 202500)            0         
_________________________________________________________________
dense (Dense)                (None, 100)               20250100  
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dropout (Dropout)            (None, 100)              

  WARNING  | None


Model: "encoder_logvar"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_logvar_inputs (Input [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 31, 31, 100)       2800      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 900)       810900    
_________________________________________________________________
flatten_1 (Flatten)          (None, 202500)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20250100  
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)            

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 100)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 25600)             2585600   
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 100)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 100)       90100     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 64, 64, 300)       270300    
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 64, 64, 3)         8103      
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64, 3)         1

  WARNING  | None


In [26]:
from evaluation.quantitive_metrics.psnr import psnr
#from evaluation.quantitive_metrics.ssmi import ssmi
#from evaluation.quantitive_metrics.sharp_diff import sharp_diff


In [27]:
ae.compile(loss={'x_logits': psnr})

Model: "pokemonConvVAEpsnr"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_logvar_inputs (InputLay [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
encoder_logvar (Model)          (None, 100)          21064200    encoder_logvar_inputs[0][0]      
__________________________________________________________________________________________________
tf_op_layer_mul (TensorFlowOpLa [(None, 100)]        0           encoder_logvar[1][0]             
__________________________________________________________________________________________________
tf_op_layer_Exp (TensorFlowOpLa [(None, 100)]        0           tf_op_layer_mul[0][0]            
_________________________________________________________________________________

In [28]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [29]:
progbar = NotebookPrograssBar(leave_outer=False)

In [30]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [31]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [32]:
csv_dir = os.path.join(recoding_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'..\\..\\recoding\\pokemonConvVAEpsnr\\csv_dir\\pokemonConvVAEpsnr.csv'

In [33]:
image_gen_dir = os.path.join(recoding_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [34]:
sg = SampleGeneration(latent_shape=latent_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [35]:
import numpy as np

In [36]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=50,
    epochs=int(1e6), 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True
)

KeyboardInterrupt: 